Thanks to https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training

In [1]:
!pip install efficientnet -q
!pip install ../input/multilabelkfold/threadpoolctl-2.1.0-py3-none-any.whl
!pip install ../input/multilabelkfold/iterative_stratification-0.1.6-py3-none-any.whl

Processing /kaggle/input/multilabelkfold/threadpoolctl-2.1.0-py3-none-any.whl
threadpoolctl is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/multilabelkfold/iterative_stratification-0.1.6-py3-none-any.whl


In [2]:
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
#from sklearn.model_selection import GroupKFold

In [3]:
img = plt.imread('../input/siim-data-640-orig-and-clahe/original/000a312787f2.png')
img.shape

(640, 640)

In [4]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 10.0)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [5]:
COMPETITION_NAME = "siim-data-640-orig-and-clahe"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 4
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

Running on TPU: grpc://10.0.0.2:8470
Running on 8 replicas


In [6]:
BATCH_SIZE

32

In [7]:
load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
label_cols = df.columns[1:5]

In [8]:
df.head(), df.shape

(                   id  Negative for Pneumonia  Typical Appearance  \
 0  00086460a852_study                       0                   1   
 1  000c9c05fd14_study                       0                   0   
 2  00292f8c37bd_study                       1                   0   
 3  005057b3f880_study                       1                   0   
 4  0051d9b12e72_study                       0                   0   
 
    Indeterminate Appearance  Atypical Appearance  
 0                         0                    0  
 1                         0                    1  
 2                         0                    0  
 3                         0                    0  
 4                         0                    1  ,
 (6054, 5))

In [9]:
my_df = pd.read_csv('../input/study-train-table/Train_Study_with_PNG.csv')
my_df.head()

,id,Img_Path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,New_Img_Path
0,00086460a852,../input/siim-covid19-detection/train/00086460...,0,1,0,0,./train(256*256*3)/65761e66de9f.png
1,000c9c05fd14,../input/siim-covid19-detection/train/000c9c05...,0,0,0,1,./train(256*256*3)/51759b5579bc.png
2,00292f8c37bd,../input/siim-covid19-detection/train/00292f8c...,1,0,0,0,./train(256*256*3)/f6293b1c49e2.png
3,005057b3f880,../input/siim-covid19-detection/train/005057b3...,1,0,0,0,./train(256*256*3)/3019399c31f4.png
4,0051d9b12e72,../input/siim-covid19-detection/train/0051d9b1...,0,0,0,1,./train(256*256*3)/bb4b1da810f3.png


In [10]:
def modify(x):
    return x.split('/')[-1]

my_df['Image_id'] = my_df['New_Img_Path'].apply(lambda x: modify(x))
my_df.head()

,id,Img_Path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,New_Img_Path,Image_id
0,00086460a852,../input/siim-covid19-detection/train/00086460...,0,1,0,0,./train(256*256*3)/65761e66de9f.png,65761e66de9f.png
1,000c9c05fd14,../input/siim-covid19-detection/train/000c9c05...,0,0,0,1,./train(256*256*3)/51759b5579bc.png,51759b5579bc.png
2,00292f8c37bd,../input/siim-covid19-detection/train/00292f8c...,1,0,0,0,./train(256*256*3)/f6293b1c49e2.png,f6293b1c49e2.png
3,005057b3f880,../input/siim-covid19-detection/train/005057b3...,1,0,0,0,./train(256*256*3)/3019399c31f4.png,3019399c31f4.png
4,0051d9b12e72,../input/siim-covid19-detection/train/0051d9b1...,0,0,0,1,./train(256*256*3)/bb4b1da810f3.png,bb4b1da810f3.png


In [11]:
full_data = pd.read_csv('../input/siim-all-images-boundg-boxes/SIIM_Combined_Data_with_Paths_and_Boxes(yolo).csv')
full_data.drop('id', axis=1, inplace=True)
full_data.head(2)

,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,Img_Path,dim0,dim1,split,yolo_bboxes
0,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,../input/siim-covid19-detection/train/5776db0c...,3488,4256,train,"[[0.30580357142857145, 0.4419642857142857, 0.2..."
1,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0,../input/siim-covid19-detection/train/ff0879eb...,2320,2832,train,0


In [12]:
full_data = full_data.rename(columns = {'StudyInstanceUID':'id'})
full_data.head(2)

,boxes,label,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,Img_Path,dim0,dim1,split,yolo_bboxes
0,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,../input/siim-covid19-detection/train/5776db0c...,3488,4256,train,"[[0.30580357142857145, 0.4419642857142857, 0.2..."
1,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0,../input/siim-covid19-detection/train/ff0879eb...,2320,2832,train,0


In [13]:
merged_data = pd.merge(my_df, full_data, on='id')
merged_data.head(3)

,id,Img_Path_x,Negative for Pneumonia_x,Typical Appearance_x,Indeterminate Appearance_x,Atypical Appearance_x,New_Img_Path,Image_id,boxes,label,Negative for Pneumonia_y,Typical Appearance_y,Indeterminate Appearance_y,Atypical Appearance_y,Img_Path_y,dim0,dim1,split,yolo_bboxes
0,00086460a852,../input/siim-covid19-detection/train/00086460...,0,1,0,0,./train(256*256*3)/65761e66de9f.png,65761e66de9f.png,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,0,1,0,0,../input/siim-covid19-detection/train/00086460...,2330,2783,train,"[[0.3169642857142857, 0.4107142857142857, 0.11..."
1,000c9c05fd14,../input/siim-covid19-detection/train/000c9c05...,0,0,0,1,./train(256*256*3)/51759b5579bc.png,51759b5579bc.png,NaN,none 1 0 0 1 1,0,0,0,1,../input/siim-covid19-detection/train/000c9c05...,3093,2850,train,0
2,00292f8c37bd,../input/siim-covid19-detection/train/00292f8c...,1,0,0,0,./train(256*256*3)/f6293b1c49e2.png,f6293b1c49e2.png,NaN,none 1 0 0 1 1,1,0,0,0,../input/siim-covid19-detection/train/00292f8c...,2539,3050,train,0


In [14]:
box_labels = []
for i in range(len(merged_data)):
    if merged_data['label'].iloc[i].startswith('none'):
        box_labels.append(0)
    else:
        box_labels.append(1)
merged_data['box_labels'] = box_labels
merged_data.head(5)

,id,Img_Path_x,Negative for Pneumonia_x,Typical Appearance_x,Indeterminate Appearance_x,Atypical Appearance_x,New_Img_Path,Image_id,boxes,label,Negative for Pneumonia_y,Typical Appearance_y,Indeterminate Appearance_y,Atypical Appearance_y,Img_Path_y,dim0,dim1,split,yolo_bboxes,box_labels
0,00086460a852,../input/siim-covid19-detection/train/00086460...,0,1,0,0,./train(256*256*3)/65761e66de9f.png,65761e66de9f.png,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,0,1,0,0,../input/siim-covid19-detection/train/00086460...,2330,2783,train,"[[0.3169642857142857, 0.4107142857142857, 0.11...",1
1,000c9c05fd14,../input/siim-covid19-detection/train/000c9c05...,0,0,0,1,./train(256*256*3)/51759b5579bc.png,51759b5579bc.png,NaN,none 1 0 0 1 1,0,0,0,1,../input/siim-covid19-detection/train/000c9c05...,3093,2850,train,0,0
2,00292f8c37bd,../input/siim-covid19-detection/train/00292f8c...,1,0,0,0,./train(256*256*3)/f6293b1c49e2.png,f6293b1c49e2.png,NaN,none 1 0 0 1 1,1,0,0,0,../input/siim-covid19-detection/train/00292f8c...,2539,3050,train,0,0
3,005057b3f880,../input/siim-covid19-detection/train/005057b3...,1,0,0,0,./train(256*256*3)/3019399c31f4.png,3019399c31f4.png,NaN,none 1 0 0 1 1,1,0,0,0,../input/siim-covid19-detection/train/005057b3...,3052,3052,train,0,0
4,0051d9b12e72,../input/siim-covid19-detection/train/0051d9b1...,0,0,0,1,./train(256*256*3)/bb4b1da810f3.png,bb4b1da810f3.png,"[{'x': 812.54698, 'y': 1376.41291, 'width': 62...",opacity 1 812.54698 1376.41291 1435.14793 1806...,0,0,0,1,../input/siim-covid19-detection/train/0051d9b1...,2336,2836,train,"[[0.3950892857142857, 0.6808035714285714, 0.21...",1


# 2 class training

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(merged_data['Image_id'], merged_data['box_labels'], test_size = 0.15, 
                                                  stratify = merged_data['box_labels'], random_state=123)

In [ ]:
valid_paths = GCS_DS_PATH + '/original/' + X_val #+ '.png' #"/train/"
train_paths = GCS_DS_PATH + '/original/' + X_train #+ '.png' #"/train/" 
valid_labels = y_val.values
train_labels = y_train.values

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 608)
IMS = 8

decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

n_labels = 1

with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(n_labels, activation='sigmoid')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(multi_label=True)])

    model.summary()


steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'B7_orig_model_608_class2.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

history = model.fit(
    train_dataset, 
    epochs=30,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

hist_df = pd.DataFrame(history.history)
hist_df.to_csv(f'history{i}.csv')

# -----------------------------------------------------------------------------------------

In [17]:
#mskf  = MultilabelStratifiedKFold(n_splits = 7)
#df['fold'] = -1
#for fold, (train_idx, val_idx) in enumerate(mskf.split(my_df['Image_id'], my_df[label_cols])):
#    my_df.loc[val_idx, 'fold'] = fold
    #print(len(train_idx), len(val_idx))
    #break

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=False, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [15]:
mskf  = MultilabelStratifiedKFold(n_splits = 7, shuffle=True, random_state=123)

for i, (train_idx, val_idx) in enumerate(mskf.split(my_df['Image_id'], my_df[label_cols])):
    
    if i == 6:
        break
    
    temp_tr = my_df.iloc[train_idx]['Image_id']
    temp_val = my_df.iloc[val_idx]['Image_id']

    valid_paths = GCS_DS_PATH + '/original/' + temp_val
    train_paths = GCS_DS_PATH + '/original/' + temp_tr
    valid_labels = my_df.iloc[val_idx][label_cols].values
    train_labels = my_df.iloc[train_idx][label_cols].values
    
    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 608, 640)
    IMS = 9

    decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

    train_dataset = build_dataset(
        train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
    )

    valid_dataset = build_dataset(
        valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
        repeat=False, shuffle=False, augment=False
    )

    try:
        n_labels = train_labels.shape[1]
    except:
        n_labels = 1

    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(n_labels, activation='softmax')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='categorical_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True)])

        model.summary()


    steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'B7_orig_model_640_{i}.h5', save_best_only=True, monitor='val_loss', mode='min')
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

    history = model.fit(
        train_dataset, 
        epochs=20,
        verbose=1,
        callbacks=[checkpoint, lr_reducer],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset)

    hist_df = pd.DataFrame(history.history)
    hist_df.to_csv(f'history{i}.csv')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=123 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


258441216/258434480 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 20, 20, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 10244     
Total params: 64,107,924
Trainable params: 63,797,204
Non-trainable params: 310,720
_________________________________________________________________
Epoch 1/20
162/162 [==============================] - 450s 1s/step - loss: 1.2781 - auc: 0.5700 - val_loss: 2.0735 - val_auc: 0.6487
Epoch 2/20
162/162 [==============================] - 89s 551ms/step - loss: 1.0956 - auc: 0.6529 - val_loss: 1.0401 - val_auc: 0.7445
Epoch 3/20
162/162 [=============

In [ ]:
for i in range(6):
        
    if i == 6:
        break
    print('\n\nSPLIT: ', i+1)
    valid_paths = GCS_DS_PATH + '/original/' + my_df[my_df['fold'] == i]['Image_id'] #+ '.png' #"/train/"
    train_paths = GCS_DS_PATH + '/original/' + my_df[my_df['fold'] != i]['Image_id'] #+ '.png' #"/train/" 
    valid_labels = my_df[my_df['fold'] == i][label_cols].values
    train_labels = my_df[my_df['fold'] != i][label_cols].values

    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 608, 640)
    IMS = 9

    decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

    train_dataset = build_dataset(
        train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder
    )

    valid_dataset = build_dataset(
        valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
        repeat=False, shuffle=False, augment=False
    )

    try:
        n_labels = train_labels.shape[1]
    except:
        n_labels = 1

    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(n_labels, activation='softmax')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='categorical_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True)])

        model.summary()


    steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'B7_orig_model_608_{i}.h5', save_best_only=True, monitor='val_loss', mode='min')
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

    history = model.fit(
        train_dataset, 
        epochs=20,
        verbose=1,
        callbacks=[checkpoint, lr_reducer],
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_dataset)

    hist_df = pd.DataFrame(history.history)
    hist_df.to_csv(f'history{i}.csv')

In [ ]:
i = 4

In [ ]:
print('\n\nSPLIT: ', i+1)
valid_paths = GCS_DS_PATH + '/orig_clahe_morph/' + my_df[my_df['fold'] == i]['Image_id'] #+ '.png' #"/train/"
train_paths = GCS_DS_PATH + '/orig_clahe_morph/' + my_df[my_df['fold'] != i]['Image_id'] #+ '.png' #"/train/" 
valid_labels = my_df[my_df['fold'] == i][label_cols].values
train_labels = my_df[my_df['fold'] != i][label_cols].values

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 608)
IMS = 8

decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]), ext='png')
test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]),ext='png')

train_dataset = build_dataset(
    train_paths, train_labels, bsize=64, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=64, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

try:
    n_labels = train_labels.shape[1]
except:
    n_labels = 1

with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB7(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(n_labels, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=[tf.keras.metrics.AUC(multi_label=True)])

    model.summary()


steps_per_epoch = train_paths.shape[0] // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'B7_orig_clahe_morph_model_608_{i}.h5', save_best_only=True, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

history = model.fit(
    train_dataset, 
    epochs=20,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

hist_df = pd.DataFrame(history.history)
hist_df.to_csv(f'history{i}.csv')